In [24]:
import sys
sys.path.insert(0, '../')
%load_ext autoreload
%autoreload 2
from lib.utils import check_exists, run_python, run_matlab, solver_name_to_n_iter
import os
import itertools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Settings

## Optimization Problem Setting

In [25]:
percNonUVal_list = [95]
budget_list = [10000, 2000, 800, 200]
VOT_list = [2.63, 1.315] 
solver_list = ["ADMM", "Gurobi", "Mosek"] # ADMM is Algorithm 1
n_iter_ADMM = 15000
config_filename = "../data/YAML/region_y3_div.yaml"
n_iter_UE = 250
n_time = 36
n_time_inc_start = 13
n_time_inc_end = 24
step_size_UE = 0.01
region_ = "region_y3_div"
setting_region = "6_9_AVG5_th1_pad_MultipleTimes"
fileName0 = "_StartHour_6_AVG5_pad_theta1e+00"
folderName0 = "Mar2May_2018_new_5-22_"
fairness = "0_0_0_100_0" # Corresponds to RHSMultiplier = [1, 1.1, 1.5, 2, 2.5]
n_companies_ADMM = 1
rho = 20
solve_binarization = 1 # For solvers other than Algorithm 1: 1=True, 0=False
seed_solver = 2 # Seed for solvers other than Algorithm 1
seed_ADMM = 2 # Seed for Algorithm 1
seed_data = 2 # Seed for generating data (e.g., selecting users from drivers)
MIPGap = 0.01 # Accuracy of the solver

## Incentivizaiton Cost Computation Setting 

In [19]:
# n_companies_cost_list = [1, 10, 100, 1000] # Number of companies in cost computation 
min_n_companies = 1 # The min number of organizations in the cost computation
max_n_companies = 10000 # The max number of organizations in the cost computation
# We compute the cost of incentivization in two settings. 
# Setting 1: number of organization is:
# range(min_n_companies, step_n_companies, max_n_companies) + {# of drivers OR individual drivers}
# Setting 1 is set if factor_n_companies=0. Setting 1 is much more time consuming but more details comapred to setting 2.
step_n_companies = 10 
# Setting 2: number of organization is:
# {min_n_companies, min_n_companies*factor_n_companies, ..., max_n_companies, # of drivers OR individual drivers}
# If factor_n_companies=-1, 1,2,3, and 5 companies will be created
factor_n_companies = 10
# Number of random sampling of drivers to companies and computing cost of incentivization
n_sample = 100

# Solving the problem


In [ ]:
for percNonUVal, VOT, solver_name, budget in itertools.product(percNonUVal_list, VOT_list, solver_list, budget_list):
    if (percNonUVal, budget, VOT, solver_name) in skip_set:
        continue
    n_iter_algo = n_iter_ADMM if solver_name == "ADMM" else solver_name_to_n_iter(solver_name)
    ################################################################################################################
    # Solving the problem
    if solver_name == "ADMM":
        # Run Algorithm 1
        ADMM_input_list = [n_iter_UE, \
                           percNonUVal, \
                           budget, \
                           n_iter_algo, \
                           n_time, \
                           n_time_inc_start, \
                           n_time_inc_end, \
                           VOT, \
                           seed_ADMM, \
                           seed_data, \
                           rho, \
                           fairness, \
                           step_size_UE, \
                           region_, \
                           setting_region, \
                           fileName0, \
                           folderName0, \
                          ]
        run_matlab(matlab_function = "addpath('../incentivization/');ADMM", 
                    matlab_input_list = ADMM_input_list, 
                    nojvm = True,
            )
        
        # Find binary solution to the provided solution by Algorithm 1 
        ILP_input_list = [n_iter_UE, \
                           percNonUVal, \
                           budget, \
                           n_iter_algo, \
                           n_time, \
                           n_time_inc_start, \
                           n_time_inc_end, \
                           VOT, \
                           seed_ADMM, \
                           seed_data, \
                           rho, \
                           fairness, \
                           step_size_UE, \
                           region_, \
                           setting_region, \
                           MIPGap, \
                          ]
        run_matlab(matlab_function = "addpath('../incentivization/');ILP", 
                    matlab_input_list = ILP_input_list, 
                    nojvm = True,
            )
    else:
        # Solve the MIP optimization problem directly using a solver
        solver_input_list = [solver_name, \
                             MIPGap, \
                             solve_binarization, \
                               n_time, \
                               n_time_inc_start, \
                               n_time_inc_end, \
                               seed_data, \
                               seed_solver, \
                               percNonUVal, \
                               budget, \
                               VOT, \
                               fairness, \
                               n_iter_UE, \
                               step_size_UE, \
                               region_, \
                               setting_region, \
                               fileName0, \
                               folderName0]
        run_matlab(matlab_function = "addpath('../incentivization/');solver_incentivization", 
                    matlab_input_list = solver_input_list, 
                    nojvm = True,
                )
    

Running command:  try;addpath('../incentivization/');solver_incentivization('Gurobi',0.01,1,36,13,24,2,2,75,10000,2.63,'0_0_0_100_0',250,0.01,'region_y3_div','6_9_AVG5_th1_pad_MultipleTimes','_StartHour_6_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  try;addpath('../incentivization/');solver_incentivization('Gurobi',0.01,1,36,13,24,2,2,75,2000,2.63,'0_0_0_100_0',250,0.01,'region_y3_div','6_9_AVG5_th1_pad_MultipleTimes','_StartHour_6_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  try;addpath('../incentivization/');solver_incentivization('Gurobi',0.01,1,36,13,24,2,2,75,800,2.63,'0_0_0_100_0',250,0.01,'region_y3_div','6_9_AVG5_th1_pad_MultipleTimes','_StartHour_6_AVG5_pad_theta1e+00','Mar2May_2018_new_5-22_');catch;end;quit
Running command:  try;addpath('../incentivization/');solver_incentivization('Gurobi',0.01,1,36,13,24,2,2,75,200,2.63,'0_0_0_100_0',250,0.01,'region_y3_div','6_9_AVG5_th1_pad_MultipleTimes','_StartHour_6

# Compute the incentivization cost

In [ ]:
for percNonUVal, VOT, solver_name, budget in itertools.product(percNonUVal_list, VOT_list, solver_list, budget_list):
    if (percNonUVal, budget, VOT, solver_name) in skip_set:
        continue
    n_iter_algo = n_iter_ADMM if solver_name == "ADMM" else solver_name_to_n_iter(solver_name)
        
    # 1. Get the volume and speed based on the post-incentivization traffic
    getSV_realCost_input_list = [n_iter_UE, \
                                 percNonUVal, \
                                 budget, \
                                 n_companies_ADMM, \
                                 n_iter_algo, \
                                 n_time, \
                                 n_time_inc_start, \
                                 n_time_inc_end, \
                                 fairness, \
                                 VOT, \
                                 seed_ADMM, \
                                 seed_data, \
                                 rho, \
                                 step_size_UE,\
                                 region_, \
                                 setting_region, \
                                 MIPGap, ]
    run_matlab(matlab_function = "addpath('../incentivization/');getSV_realCost", 
                matlab_input_list = getSV_realCost_input_list, 
                nojvm = False,
            )

    # 2. Compute the R matrix and other info based on new speed and volume
    run_python(py_version = 2, 
               py_file = "../incentivization/run_realCost.py", 
               arg_names = ["config_filename", \
                            "sA", \
                            "sD", \
                            "nC", \
                            "f", \
                            "percNonU", \
                            "iterRun", \
                            "b", \
                            "VOT", \
                            "nTIS", \
                            "nTIE", \
                            "it", \
                            "MIPGap", \
                           ], 
               arg_values = [config_filename, \
                             seed_ADMM, \
                             seed_data, \
                             n_companies_ADMM, \
                             fairness, \
                             percNonUVal, \
                             n_iter_UE, \
                             budget, \
                             VOT, \
                             n_time_inc_start, \
                             n_time_inc_end, \
                             n_iter_algo, \
                             MIPGap, \
                            ],
              )

    # 3. Compute the incentivization cost
    compareCosts_realCost_initAll2_input_list = [n_iter_UE, \
                                                 percNonUVal, \
                                                 budget, \
                                                 n_companies_ADMM, \
                                                 n_iter_algo, \
                                                 min_n_companies, \
                                                 max_n_companies, \
                                                 step_n_companies, \
                                                 factor_n_companies, \
                                                 # Number of samples of cost computation, randomness is in selecting companies' drivers
                                                 n_sample, \
                                                 n_time, \
                                                 n_time_inc_start, \
                                                 n_time_inc_end, \
                                                 fairness, \
                                                 VOT, \
                                                 seed_ADMM, \
                                                 seed_data, \
                                                 rho, \
                                                 step_size_UE,\
                                                 region_, \
                                                 setting_region, \
                                                 MIPGap, \
                                                ]
    run_matlab(matlab_function = "addpath('../incentivization/');compareCosts_realCost_initAll2_allInOne", 
                matlab_input_list = compareCosts_realCost_initAll2_input_list, 
                nojvm = True,
            )

